# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [2]:
# import required libraries
import pandas as pd
import numpy as np

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [3]:
# your code here

orders = pd.read_csv('orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


In [4]:
orders.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397924 entries, 0 to 397923
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    397924 non-null  int64  
 1   InvoiceNo     397924 non-null  int64  
 2   StockCode     397924 non-null  object 
 3   year          397924 non-null  int64  
 4   month         397924 non-null  int64  
 5   day           397924 non-null  int64  
 6   hour          397924 non-null  int64  
 7   Description   397924 non-null  object 
 8   Quantity      397924 non-null  int64  
 9   InvoiceDate   397924 non-null  object 
 10  UnitPrice     397924 non-null  float64
 11  CustomerID    397924 non-null  int64  
 12  Country       397924 non-null  object 
 13  amount_spent  397924 non-null  float64
dtypes: float64(2), int64(8), object(4)
memory usage: 42.5+ MB


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# your code here



In [5]:
#amount_spend_unique = orders.groupby('CustomerID')['amount_spent'].sum()

customers_orders = orders.groupby('CustomerID').agg({'amount_spent': 'sum'})
customers_orders.head()



,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


In [6]:
percentil_95 = np.percentile(customers_orders['amount_spent'],95)
percentil_75 = np.percentile(customers_orders['amount_spent'],75)



In [7]:
customers_orders['tipo_cliente'] = customers_orders['amount_spent'].apply(lambda x: 'vip' if x > percentil_95 else 'preferred' if x < percentil_95 and x > percentil_75 else 'normal')
                                                                          
customers_orders.head()

,amount_spent,tipo_cliente
CustomerID,,
12346,77183.60,vip
12347,4310.00,preferred
12348,1797.24,preferred
12349,1757.55,preferred
12350,334.40,normal


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [8]:
# your code here


In [24]:
customers_orders['Country'] = orders['Country']



customers_orders


estatus = pd.get_dummies(customers_orders['tipo_cliente'])

estatus['Country'] = customers_orders['Country']
final = estatus.groupby('Country').agg({'vip':'sum', 'preferred':'sum', 'normal':'sum'})
final.sort_values(by='vip', ascending=False)

final['vip + pref'] = final['vip'] + final['preferred']

final


,vip,preferred,normal,vip + pref
Country,,,,
Australia,1,4.0,9.0,5.0
EIRE,2,8.0,15.0,10.0
France,0,8.0,19.0,8.0
Germany,0,7.0,29.0,7.0
Netherlands,0,0.0,2.0,0.0
Norway,9,8.0,56.0,17.0
Spain,1,1.0,3.0,2.0
Switzerland,0,1.0,5.0,1.0
United Kingdom,204,831.0,3116.0,1035.0


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [10]:
# your code here


In [20]:
vip_pais = customers_orders.groupby('tipo_cliente').Country.value_counts()




In [21]:
vip_pais




tipo_cliente  Country       
normal        United Kingdom    3116
              Norway              56
              Germany             29
              France              19
              EIRE                15
              Australia            9
              Switzerland          5
              Spain                3
              Netherlands          2
preferred     United Kingdom     831
              EIRE                 8
              France               8
              Norway               8
              Germany              7
              Australia            4
              Spain                1
              Switzerland          1
vip           United Kingdom     204
              Norway               9
              EIRE                 2
              Australia            1
              Spain                1
Name: Country, dtype: int64